In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import string
import itertools

import numpy as np

In [3]:
import random_indexing as ri

In [4]:
features = ri.Features(list('abcde ') + [''], 10)

In [5]:
_a = features[''][features.permutation] * features['a']

ab = features['a'][features.permutation] * features['b']
bc = features['b'][features.permutation] * features['c']
cd = features['c'][features.permutation] * features['d']

c_ = features['c'][features.permutation] * features['']
d_ = features['d'][features.permutation] * features['']

In [6]:
assert (ab == ab * bc * bc).all()

In [7]:
_ab = features[''][features.permutation][features.permutation] * ab

abc = features['a'][features.permutation][features.permutation] * bc
bcd = features['b'][features.permutation][features.permutation] * cd

cd_ = features['c'][features.permutation][features.permutation] * d_

In [8]:
_abc = (
    features[''][features.permutation][features.permutation][features.permutation] * 
    abc
)

abcd = (
    features['a'][features.permutation][features.permutation][features.permutation] * 
    bcd
)

In [10]:
assert (
    np.stack(ri.vectorize_naive('abc', window_length=2, features=features)) ==
    np.stack([_a, ab, bc, c_])
).all()

In [11]:
assert (
    np.stack(ri.vectorize('abc', window_length=2, features=features)) ==
    np.stack([_a, ab, bc, c_])
).all()

In [12]:
assert (
    np.stack(ri.vectorize('abcd', window_length=3, features=features)) ==
    np.stack([_ab, abc, bcd, cd_])
).all()

In [14]:
t1 = np.stack(ri.vectorize('aabbccddeeaabbcceedd', window_length=5, features=features))

In [15]:
t1n = np.stack(ri.vectorize_naive('aabbccddeeaabbcceedd', window_length=5, features=features))

In [16]:
_aabb = (
    features['' ][features.permutation][features.permutation][features.permutation][features.permutation] *
    features['a'][features.permutation][features.permutation][features.permutation] *
    features['a'][features.permutation][features.permutation] *
    features['b'][features.permutation] *
    features['b']
)

In [17]:
assert (t1[0] == _aabb).all()

In [18]:
assert (t1n[0] == _aabb).all()

In [19]:
assert (t1 == t1n).all()

In [20]:
def LongText(features, N):
    return itertools.islice(itertools.cycle(features), N)

In [21]:
def test(vectorizer, N, window_lenght, features):
    return sum(
        vectorizer(
            LongText(features.features, N),
            window_lenght, features,
        )
    )

In [23]:
features10K = ri.Features(list(string.ascii_lowercase) + [''], 10_000)

In [24]:
for _N in 1_000, 10_000, 100_000:
    print(_N)

    %time _ = test(ri.vectorize, _N, 5, features10K)
    %time _naive = test(ri.vectorize_naive, _N, 5, features10K)
    
    assert (_ == _naive).all()

1000
CPU times: user 83.6 ms, sys: 2.44 ms, total: 86 ms
Wall time: 85 ms
CPU times: user 144 ms, sys: 788 µs, total: 145 ms
Wall time: 145 ms
10000
CPU times: user 849 ms, sys: 4.98 ms, total: 854 ms
Wall time: 873 ms
CPU times: user 1.79 s, sys: 15.3 ms, total: 1.81 s
Wall time: 1.93 s
100000
CPU times: user 8.64 s, sys: 57.6 ms, total: 8.7 s
Wall time: 8.83 s
CPU times: user 17.8 s, sys: 139 ms, total: 17.9 s
Wall time: 18.4 s
